In [1]:
!git clone https://github.com/SobhanRjz/DL_Detectron2.git
!git pull --force
%cd DL_Detectron2

fatal: destination path 'DL_Detectron2' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
/content/DL_Detectron2


In [2]:
!git pull --force
!apt-get install unrar

Already up to date.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [3]:
# prompt: makedir if not exist /content/DL_Detectron2/DataSets/images
import os
# Create the directory if it does not exist
dir_path = "/content/DL_Detectron2/DataSets/images"
if not os.path.exists(dir_path):

    os.makedirs(dir_path)
    print(f"Directory '{dir_path}' created successfully.")
else:
    print(f"Directory '{dir_path}' already exists.")

Directory '/content/DL_Detectron2/DataSets/images' already exists.


In [4]:
# prompt: get from drive folder

from google.colab import drive
drive.mount('/content/drive')
# Then you can use standard file operations to access the files.
# For example, to list the files in a folder:
!ls "/content/drive/My Drive/Colab Notebooks"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
images.part1.rar  images.part5.rar  output.zip	     Untitled3.ipynb
images.part2.rar  images.part6.rar  Untitled0.ipynb  Untitled4.ipynb
images.part3.rar  images.rar	    Untitled1.ipynb
images.part4.rar  model_final.pth   Untitled2.ipynb


In [5]:
!unrar x "/content/drive/MyDrive/Colab Notebooks/images.part1.rar" "/content/DL_Detectron2/DataSets/images"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/Colab Notebooks/images.part1.rar


Would you like to replace the existing file /content/DL_Detectron2/DataSets/images/train/00001_jpg.rf.e949ddc12765f117d9c60c6bc511e819.jpg
  5210 bytes, modified on 2025-04-22 08:58
with a new one
  5210 bytes, modified on 2025-04-22 08:58

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit Q

Program aborted


In [5]:

# Look at training curves in tensorboard:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

from google.colab.patches import cv2_imshow



  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
fatal: destination path 'detectron2' already exists and is not an empty directory.
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.6 ; cuda:  cu124
detectron2: 0.6


In [6]:
import logging
import sys

# Configure the logger
logging.basicConfig(
    level=logging.INFO,  # Set the logging level to INFO
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True, # Resets any previous configuration
    handlers=[
        logging.StreamHandler(sys.stdout)  # Stream logs to Colab's cell output
    ]
)

# Create logger instance
logger = logging.getLogger()
logger.info("Logger is configured!")


2025-04-24 13:11:25,149 [INFO] Logger is configured!


In [7]:
import os
import sys
# Get the parent directory path
parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


import logging
from pathlib import Path
import torch
from Config.basic_config import BASE_PATH, DATA_PATH, OUTPUT_PATH, DEVICE
from Config.basic_config import detectron2_logger as logger
from Config.detectron2_config import DetectronConfig
from Config.dataset_config import DatasetConfig
from PreProcess.preprocess_COCO import COCOJsonProcessor
from Train import mainTrain
from detectron2.engine import launch


In [8]:
def process_datasets():
    """Handle dataset preprocessing and registration"""
    # Use pathlib for more robust path handling
    json_paths = [
        Path(DATA_PATH) / split / "_annotations.coco.json"
        for split in ["train", "test", "valid"]
    ]

    logger.info("Processing COCO JSON files...")
    # coco_processor = COCOJsonProcessor([str(p) for p in json_paths])
    # coco_processor.process_files()



    logger.info("Registering datasets...")
    DatasetConfig().register_datasets()

    from collections import Counter
    from detectron2.data import DatasetCatalog
    import matplotlib.pyplot as plt
    import matplotlib

    #matplotlib.use('TkAgg')  # Use Tkinter backend
    # Load dataset
    dataset_dicts = DatasetCatalog.get("my_dataset_train")

    # Count instances per class
    class_counts = Counter()
    for data in dataset_dicts:
        for annotation in data["annotations"]:
            class_id = annotation["category_id"]
            class_counts[class_id] += 1

    # Create bar plot
    # plt.figure(figsize=(10, 6))
    # classes = list(class_counts.keys())
    # counts = list(class_counts.values())

    # plt.bar(classes, counts)
    # plt.title('Distribution of Classes in Training Dataset')
    # plt.xlabel('Class ID')
    # plt.ylabel('Number of Instances')

    # # Add count labels on top of each bar
    # for i, count in enumerate(counts):
    #     plt.text(classes[i], count, str(count), ha='center', va='bottom')

    # plt.tight_layout()
    # plt.savefig(str(Path(OUTPUT_PATH) / 'class_distribution.png'))
    # plt.close()

    # print("Class Distribution:", class_counts)
    # print("Distribution plot saved to:", str(Path(OUTPUT_PATH) / 'class_distribution.png'))


def setup_training():
    """Configure training settings and get GPU count"""
    logger.info("Loading Detectron2 configuration...")


    detectron_config = DetectronConfig()
    cfg = detectron_config.get_cfg()

    num_gpus = torch.cuda.device_count()
    logger.info(f"Number of GPUs available: {num_gpus}")

    return cfg, num_gpus


In [9]:
"""Main execution function"""
logger.info("Initializing project...")

# Process and register datasets
process_datasets()

# Setup training configuration
cfg, num_gpus = setup_training()


2025-04-24 13:11:31,573 [INFO] Initializing project...
2025-04-24 13:11:31,574 [INFO] Processing COCO JSON files...
2025-04-24 13:11:31,575 [INFO] Registering datasets...
['coco_2014_train', 'coco_2014_val', 'coco_2014_minival', 'coco_2014_valminusminival', 'coco_2017_train', 'coco_2017_val', 'coco_2017_test', 'coco_2017_test-dev', 'coco_2017_val_100', 'keypoints_coco_2014_train', 'keypoints_coco_2014_val', 'keypoints_coco_2014_minival', 'keypoints_coco_2014_valminusminival', 'keypoints_coco_2017_train', 'keypoints_coco_2017_val', 'keypoints_coco_2017_val_100', 'coco_2017_train_panoptic_separated', 'coco_2017_train_panoptic_stuffonly', 'coco_2017_train_panoptic', 'coco_2017_val_panoptic_separated', 'coco_2017_val_panoptic_stuffonly', 'coco_2017_val_panoptic', 'coco_2017_val_100_panoptic_separated', 'coco_2017_val_100_panoptic_stuffonly', 'coco_2017_val_100_panoptic', 'lvis_v1_train', 'lvis_v1_val', 'lvis_v1_test_dev', 'lvis_v1_test_challenge', 'lvis_v0.5_train', 'lvis_v0.5_val', 'lvis_

Processing images: 100%|██████████| 10567/10567 [02:35<00:00, 68.16it/s]

Total Iterations: ********************************* 50000 ***************************************
Configuration saved to /content/DL_Detectron2/output/mask_rcnn_X_101_32x8d_FPN_3x.yaml
2025-04-24 13:14:19,226 [INFO] Number of GPUs available: 1


In [10]:
# Launch distributed training
logger.info("Launching training...")
trained_model = launch(
    mainTrain,
    num_gpus,
    num_machines=1,
    machine_rank=0,
    dist_url="auto",
    args=(cfg, "default", False),
)
# Save final model
logger.info("Saving final model...")
# final_model_path = os.path.join(OUTPUT_PATH, "model_final.pth")
# torch.save(trained_model.state_dict(), final_model_path)  # Save the trained model
# logger.info(f"Final model saved to: {}")
logger.info(f"Configuration loaded successfully with device: {DEVICE}")

2025-04-24 13:14:21,006 [INFO] Launching training...
2025-04-24 13:14:21,007 [INFO] Initializing default trainer...
2025-04-24 13:14:22,771 [INFO] Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(

/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2025-04-24 13:15:11,037 [INFO]  iter: 19  total_loss: 4.472  loss_cls: 2.427  loss_box_reg: 0.09123  loss_mask: 0.692  loss_rpn_cls: 0.6854  loss_rpn_loc: 0.591     lr: 1.9981e-05  max_mem: 16329M
2025-04-24 13:15:48,735 [INFO]  eta: 1 day, 2:09:29  iter: 39  total_loss: 3.177  loss_cls: 1.035  loss_box_reg: 0.07376  loss_mask: 0.6902  loss_rpn_cls: 0.6818  loss_rpn_loc: 0.5936     lr: 3.9961e-05  max_mem: 16329M


KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()